### Imports

In [1]:
import gym
import numpy as np
import random
import collections
import keras

Using TensorFlow backend.


#### Set up agent environment

In [ ]:
def cartpole():
    env = gym.make('CartPole-v0')
    obs_space = env.observation_space.shape[0]
    action_space = env.action_space.n

    while True:
        state = env.reset()